In [ ]:
# default_exp scrape13G

# scrape13G

> Scrape holdings information from 13G SEC filings.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

import collections
import itertools
import os
import re

from secscan import utils, dailyList, basicInfo, infoScraper

default13GDir = os.path.join(utils.stockDataRoot,'scraped13G')

13G scraper class - scrape holdings information from the SEC filing:

In [ ]:
#export

nSharesPatStr = r'(\d+(?:[,.]\d\d\d)*)'
nPctPatStr = r'(\d+(?:\.\d*)?|\.\d+)'
form13NshAndPctPats = [
    re.compile(r'aggregate\s+amount.{1,100}?' + nSharesPatStr
                + r'.{1,200}?' + r'percent\s+of\s+class.{1,100}?' + nPctPatStr + r'\s*%',
                re.IGNORECASE|re.DOTALL),
    re.compile(r'item\s+9\s*:.*?' + nSharesPatStr
                + r'.*?' + r'item\s+11\s*:.*?' + nPctPatStr + r'\s*%',
                re.IGNORECASE|re.DOTALL),
    re.compile(r'aggregate\s+amount.{1,100}?' + nSharesPatStr
                + r'.{1,200}?' + r'percent\s+of class.{1,100}?(?!\D9\D)\D' + nPctPatStr,
                re.IGNORECASE|re.DOTALL),
]
def getSec13NshAndPctFromText(txt) :
    "Returns a list [(nShares, percent) ... ] parsed from form 13G or 13D."
    for pat in form13NshAndPctPats :
        res = pat.findall(txt)
        if res :
            break
    return res

purposePat = re.compile(r'4\s*\.?\s*purpose\s*of\s*(?:the\s*)?transaction(?:\s*\.?\s*)(.{1,10000}?)'
                        + r'(?:\s*(?:item\s*)?5\s*\.?\s*interest'
                            + r'|\s*(?:item\s*)?6\s*\.?\s*contracts'
                            + r'|\s*(?:item\s*)?7\s*\.?\s*material'
                            + r'|\s*after\s*reasonable\s*inquiry'
                            + r'|\s*$'
                        + r')',
                        re.IGNORECASE|re.DOTALL)

def cusipChecksum(cusip) :
    s = 0
    for i,c in enumerate(cusip[:8]) :
        if c.isdigit() :
            v = ord(c) - ord('0')
        elif c.isalpha() :
            v = 10 + ord(c.upper()) - ord('A')
        if (i&1) == 1 :
            v *= 2
        s += (v//10) + (v%10)
    return str((10 - (s%10)) % 10)

cusipPatStr = r'[0-9A-Za-z]\d[0-9A-Za-z]\d[0-9A-Za-z]{2}(?:[-_\s]*[0-9A-Za-z]{2}(?:[-_\s]*\d)?)?'
cusipNumberPatStr = r'cusip\s*(?:number|#|no)'
cusipSearchPat1 = re.compile(r'.{1,3000}?\s+(' + cusipPatStr + r')\s+.{0,200}?\s*' + cusipNumberPatStr,
                             re.IGNORECASE|re.DOTALL)
cusipSearchPat2 = re.compile(r'.{1,3000}?\s*' + cusipNumberPatStr + '.{0,200}?\s*(' + cusipPatStr + r')\s+',
                             re.IGNORECASE|re.DOTALL)
cusipSearchPat3 = re.compile(r'.{1,2000}\n\s*(' + cusipPatStr + r')\s*\n',
                             re.IGNORECASE|re.DOTALL)
spaceDashPat = re.compile(r'[-\s]*')

def parse13GD(accNo, formType=None) :
    info = basicInfo.getSecFormInfo(accNo, formType=formType)
    links = info['links']
    if len(links) == 0 :
        print('NO LINKS LIST!')
        info['positions'] = []
    else :
        toFormat = 'text' if links[0][3].endswith('.txt') else 'souptext'
        mainText = utils.downloadSecUrl(links[0][3], toFormat=toFormat)
        info['positions'] = getSec13NshAndPctFromText(mainText)
        m = cusipSearchPat1.match(mainText)
        if m is None :
            m = cusipSearchPat2.match(mainText)
        if m is None and toFormat=='text' :
            m = cusipSearchPat3.match(mainText)
        if m is None :
            if '0001504304' not in info['ciks'] :
                # suppress the message for 0001504304 - Bulldog Investors
                # - they don't report CUSIPs in their filings
                print('no CUSIP found!')
        else :
            cusip = spaceDashPat.sub('',m.group(1))
            if len(cusip) == 6 :
                print('adding 10 to CUSIP',cusip)
                cusip = cusip + '10'
            if len(cusip) == 8 :
                print('adding checksum to CUSIP',cusip)
                cusip = cusip + cusipChecksum(cusip)
            if len(cusip)!=9 or cusip[8]!=cusipChecksum(cusip) :
                print('invalid CUSIP!',cusip)
            info['cusip'] = cusip
            # print('CUSIP-'+cusip,end=' ')
        if formType is None :
            formType = links[0][2]
        if formType.upper().startswith('SC 13D') :
            m = purposePat.search(mainText)
            if m is None :
                print('no purpose of transaction found!')
            else :
                info['purpose'] = m.group(1)
    if len(info['positions']) == 0 :
        print('no positions found!')
    return info #,mainText

class scraper13G(infoScraper.scraperBase) :
    def __init__(self, infoDir=default13GDir, startD=None, endD=None, fSuff='m.pkl', **pickle_kwargs) :
        super().__init__(infoDir, 'SC 13G', startD=startD, endD=endD, fSuff=fSuff, **pickle_kwargs)
    def scrapeInfo(self, accNo, formType=None) :
        return parse13GD(accNo, formType=formType), None

Test 13G scraper class:

In [ ]:
dl = dailyList.dailyList(startD='empty')
dl.updateForDays('20210701','20210704')
assert len(dl.getFilingsList(None,'SC 13G')[0])==100,"testing 13G scraper class (daily list count)"
info = parse13GD('0001567619-21-013814', formType='SC 13G')
assert (info['ciks']==['0000016099', '0001373604']
        and info['positions']==[('1350552', '4.36'), ('1582235', '5.10')]
        and info['cusip']=='549282101'
    ),"testing 13G scraper class (parsing)"

20210703 WEEKEND 20210702 ### list index 28 filings for 20210702: 6569 * 20210701 filings for 20210701: 5573 * 

In [ ]:
#hide
# accNo='0001050470-19-000050'; utils.secBrowse(accNo)

In [ ]:
#hide
# parse13GD(accNo)

In [ ]:
#hide
# dailyList.dlCountFilings(startD='20210101',formClass='SC 13G')

In [ ]:
#hide
# uncomment and run to regenerate all library Python files
# from nbdev.export import notebook2script; notebook2script()